In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
app_test = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/application_test.csv.zip', compression='zip')
app_train = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/application_train.csv.zip', compression='zip')

# preprocessing 

In [10]:
print(app_test.shape, app_train.shape)

(48744, 121) (307511, 122)


In [11]:
df = app_train.append(app_test).reset_index()

/home/ir_ciambotti_gmail_com/.venv/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in df:
    if df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(df[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            df[col] = le.transform(df[col])            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [13]:
# one hot encoding
categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
df = pd.get_dummies(df, columns= categorical_columns, dummy_na= True)
print(df.dtypes.value_counts())

uint8      147
float64     66
int64       44
dtype: int64


In [14]:
#df.isnull().sum().sort_values(ascending = False).head()

# adding features from other datasets

*initially working with the first 1000 rows of each for efficiency*

In [52]:
bureau = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/bureau.csv.zip', compression='zip').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]

<h4> bureau_bal is missing SK_ID column!

In [51]:
bureau_bal = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/bureau_balance.csv.zip', compression='zip')

In [17]:
bureau_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


<h5> transform dataframe so that one row is one loan-- function copied from Will's "start here" kaggle kernel!!

In [18]:
def count_categorical(df, group_var, df_name):
    """Computes counts and normalized counts for each observation
    of `group_var` of each unique category in every categorical variable
    
    Parameters
    --------
    df : dataframe 
        The dataframe to calculate the value counts for.
        
    group_var : string
        The variable by which to group the dataframe. For each unique
        value of this variable, the final dataframe will have one row
        
    df_name : string
        Variable added to the front of column names to keep track of columns

    
    Return
    --------
    categorical : dataframe
        A dataframe with counts and normalized counts of each unique category in every categorical variable
        with one row for every unique value of the `group_var`.
        
    """
    
    # Select the categorical columns
    categorical = pd.get_dummies(df.select_dtypes('object'))

    # Make sure to put the identifying id on the column
    categorical[group_var] = df[group_var]

    # Groupby the group var and calculate the sum and mean
    categorical = categorical.groupby(group_var).agg(['sum', 'mean'])
    
    column_names = []
    
    # Iterate through the columns in level 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['count', 'count_norm']:
            # Make a new column name
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
    
    return categorical

In [19]:
bureau_bal_counts = count_categorical(bureau_bal, group_var = 'SK_ID_BUREAU', df_name = 'bureau_bal')
bureau_bal_counts.head()

,bureau_bal_STATUS_0_count,bureau_bal_STATUS_0_count_norm,bureau_bal_STATUS_1_count,bureau_bal_STATUS_1_count_norm,bureau_bal_STATUS_2_count,bureau_bal_STATUS_2_count_norm,bureau_bal_STATUS_3_count,bureau_bal_STATUS_3_count_norm,bureau_bal_STATUS_4_count,bureau_bal_STATUS_4_count_norm,bureau_bal_STATUS_5_count,bureau_bal_STATUS_5_count_norm,bureau_bal_STATUS_C_count,bureau_bal_STATUS_C_count_norm,bureau_bal_STATUS_X_count,bureau_bal_STATUS_X_count_norm
SK_ID_BUREAU,,,,,,,,,,,,,,,,
5001709,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,86,0.886598,11,0.113402
5001710,5,0.060241,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,48,0.578313,30,0.361446
5001711,3,0.750000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,1,0.250000
5001712,10,0.526316,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,9,0.473684,0,0.000000
5001713,0,0.000000,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0,0.000000,22,1.000000


In [20]:
# Merge to include the SK_ID_CURR
bureau_by_loan = bureau_bal_counts.merge(bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')

In [21]:
# retrieve the first 1,000 sorted values as usual
bureau_bal_2 = bureau_by_loan.sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]

In [22]:
cash = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/POS_CASH_balance.csv.zip', compression='zip').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
credit = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/credit_card_balance.csv.zip', compression='zip').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
previous = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/previous_application.csv.zip', compression='zip').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
installments = pd.read_csv('~/.kaggle/competitions/home-credit-default-risk/installments_payments.csv.zip', compression='zip').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]

# let's try featuretools (!?)

In [53]:
import featuretools as ft

In [55]:
# make sure featuretools knows what our variable types are
df_types = {}

# Iterate through the columns and record the Boolean columns
for col in df_train:
    # If column is a number with only two values, encode it as a Boolean
    if (df_train[col].dtype != 'object') and (len(df_train[col].unique()) <= 2):
        df_types[col] = ft.variable_types.Boolean

print('Number of boolean variables: ', len(df_types))

Number of boolean variables:  182


In [56]:
df_types['REGION_RATING_CLIENT'] = ft.variable_types.Ordinal
df_types['REGION_RATING_CLIENT_W_CITY'] = ft.variable_types.Ordinal

In [57]:
previous_types= {'NFLAG_LAST_APPL_IN_DAY': ft.variable_types.Boolean,
                 'NFLAG_INSURED_ON_APPROVAL': ft.variable_types.Boolean}

In [ ]:
df_train = df[df['TARGET'].notnull()]
y = df_train['TARGET']
df_test = df[df['TARGET'].isnull()]

del df_train['TARGET']
del df_test['TARGET']

<h3> fixing datetime

In [58]:
# fix time outliers (thanks github)
import re

def replace_day_outliers(df):
    """Replace 365243 with np.nan in any columns with DAYS"""
    for col in df.columns:
        if "DAYS" in col:
            df[col] = df[col].replace({365243: np.nan})

    return df

# Replace all the day outliers
df_train = replace_day_outliers(df_train)
df_test = replace_day_outliers(df_test)
bureau = replace_day_outliers(bureau)
bureau_bal = replace_day_outliers(bureau_bal)
credit = replace_day_outliers(credit)
cash = replace_day_outliers(cash)
previous = replace_day_outliers(previous)
installments = replace_day_outliers(installments)

In [59]:
# make arbitrary starting time
start_date = pd.Timestamp("1997-06-11")
start_date

Timestamp('1997-06-11 00:00:00')

In [60]:
# convert to timedelta (d)!
for col in ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE']:
    bureau[col] = pd.to_timedelta(bureau[col], 'D')
    
bureau[['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_ENDDATE_FACT', 'DAYS_CREDIT_UPDATE']].head()

,DAYS_CREDIT,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,DAYS_CREDIT_UPDATE
0,-1572 days,-1329 days,-1328 days,-155 days
1,-909 days,-179 days,-877 days,-155 days
2,-320 days,411 days,NaT,-10 days
3,-49 days,1778 days,NaT,-16 days
4,-879 days,-514 days,-544 days,-155 days


In [61]:
bureau['bureau_credit_app_date'] = start_date + bureau['DAYS_CREDIT']
bureau['bureau_credit_end_date'] = start_date + bureau['DAYS_CREDIT_ENDDATE']
bureau['bureau_credit_close_date'] = start_date + bureau['DAYS_ENDDATE_FACT']
bureau['bureau_credit_update_date'] = start_date + bureau['DAYS_CREDIT_UPDATE']

In [72]:
bureau.drop(columns = ['DAYS_CREDIT', 'DAYS_CREDIT_ENDDATE', 'DAYS_CREDIT_UPDATE', 'DAYS_ENDDATE_FACT']);

In [62]:
bureau_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [63]:
bureau_bal['MONTHS_BALANCE'] = pd.to_timedelta(bureau_bal['MONTHS_BALANCE'], 'M')
bureau_bal['bureau_bal_date'] = start_date + bureau_bal['MONTHS_BALANCE']

In [70]:
bureau_bal.drop(columns = ['MONTHS_BALANCE']);

In [65]:
previous.columns

Index(['SK_ID_PREV', 'SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'AMT_ANNUITY',
       'AMT_APPLICATION', 'AMT_CREDIT', 'AMT_DOWN_PAYMENT', 'AMT_GOODS_PRICE',
       'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
       'FLAG_LAST_APPL_PER_CONTRACT', 'NFLAG_LAST_APPL_IN_DAY',
       'RATE_DOWN_PAYMENT', 'RATE_INTEREST_PRIMARY',
       'RATE_INTEREST_PRIVILEGED', 'NAME_CASH_LOAN_PURPOSE',
       'NAME_CONTRACT_STATUS', 'DAYS_DECISION', 'NAME_PAYMENT_TYPE',
       'CODE_REJECT_REASON', 'NAME_TYPE_SUITE', 'NAME_CLIENT_TYPE',
       'NAME_GOODS_CATEGORY', 'NAME_PORTFOLIO', 'NAME_PRODUCT_TYPE',
       'CHANNEL_TYPE', 'SELLERPLACE_AREA', 'NAME_SELLER_INDUSTRY',
       'CNT_PAYMENT', 'NAME_YIELD_GROUP', 'PRODUCT_COMBINATION',
       'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION',
       'DAYS_LAST_DUE', 'DAYS_TERMINATION', 'NFLAG_INSURED_ON_APPROVAL'],
      dtype='object')

In [66]:
for col in ['DAYS_DECISION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']:
    previous[col] = pd.to_timedelta(previous[col], 'D')

In [67]:
previous['previous_decision_date'] = start_date + previous['DAYS_DECISION']
previous['previous_drawing_date'] = start_date + previous['DAYS_FIRST_DRAWING']
previous['previous_first_due_date'] = start_date + previous['DAYS_FIRST_DUE']
previous['previous_last_duefirst_date'] = start_date + previous['DAYS_LAST_DUE_1ST_VERSION']
previous['previous_last_due_date'] = start_date + previous['DAYS_LAST_DUE']
previous['previous_termination_date'] = start_date + previous['DAYS_TERMINATION']

In [68]:
previous = previous.drop(columns = ['DAYS_DECISION', 'DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION'])

In [73]:
# Convert to timedelta objects
credit['MONTHS_BALANCE'] = pd.to_timedelta(credit['MONTHS_BALANCE'], 'M')
cash['MONTHS_BALANCE'] = pd.to_timedelta(cash['MONTHS_BALANCE'], 'M')

# Make a date column
credit['credit_balance_date'] = start_date + credit['MONTHS_BALANCE']
credit = credit.drop(columns = ['MONTHS_BALANCE'])

# Make a date column
cash['cash_balance_date'] = start_date + cash['MONTHS_BALANCE']
cash = cash.drop(columns = ['MONTHS_BALANCE'])

In [74]:
# Conver to time delta object
installments['DAYS_INSTALMENT'] = pd.to_timedelta(installments['DAYS_INSTALMENT'], 'D')
installments['DAYS_ENTRY_PAYMENT'] = pd.to_timedelta(installments['DAYS_ENTRY_PAYMENT'], 'D')

# Create time column and drop
installments['installments_due_date'] = start_date + installments['DAYS_INSTALMENT']
installments = installments.drop(columns = ['DAYS_INSTALMENT'])

installments['installments_paid_date'] = start_date + installments['DAYS_ENTRY_PAYMENT']
installments = installments.drop(columns = ['DAYS_ENTRY_PAYMENT'])

<h3> using featuretools

In [77]:
es = ft.EntitySet(id = 'clients')

In [79]:
# Entities with a unique index
es = es.entity_from_dataframe(entity_id = 'df_train', dataframe = df_train, 
                              index = 'SK_ID_CURR', variable_types = df_types)

es = es.entity_from_dataframe(entity_id = 'df_test', dataframe = df_test, 
                              index = 'SK_ID_CURR', variable_types = df_test_types)

es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, 
                              index = 'SK_ID_BUREAU', time_index='bureau_credit_application_date')

es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, 
                              index = 'SK_ID_PREV', time_index = 'previous_decision_date',
                              variable_types = previous_types)

# Entities that do not have a unique index
es = es.entity_from_dataframe(entity_id = 'bureau_bal', dataframe = bureau_balance, 
                              make_index = True, index = 'bb_index',
                              time_index = 'bureau_bal_date')

es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                              make_index = True, index = 'cash_index',
                              time_index = 'cash_balance_date')

es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,
                              make_index = True, index = 'install_index',
                              time_index = 'installments_paid_date')

es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                              make_index = True, index = 'credit_index',
                              time_index = 'credit_balance_date')

NameError: name 'df_test_types' is not defined

# feature selection

# cross validation

In [ ]:
#from sklearn.model_selection import StratifiedKFold

#skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=27)

#X_train, X_test, y_train, y_test = skf.split(X, y)

In [ ]:
df_train = df[df['TARGET'].notnull()]
y = df_train['TARGET']
df_test = df[df['TARGET'].isnull()]

In [ ]:
del df_train['TARGET']
del df_test['TARGET']

In [ ]:
print(df_train.shape, df_test.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_train, y, test_size=0.2, random_state=42)